<a href="https://colab.research.google.com/github/03axdov/Notebooks/blob/main/MNIST2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyyaml h5py  # Required to save models in HDF5 format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.8.2


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

print(f"TRAIN IMAGES: {train_images.shape}")
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0 # Reshaping as no conv layers are used
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11501568/11490434 [==============================] - 0s 0us/step
TRAIN IMAGES: (60000, 28, 28)


In [5]:
def create_model():
  model = tf.keras.Sequential([
      keras.layers.Dense(512, activation="relu", input_shape=(28*28,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
  ])

  model.compile(
      optimizer = tf.keras.optimizers.Adam(),
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[
          tf.keras.metrics.SparseCategoricalAccuracy()
      ]
  )

  return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint_path = "training_1/ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    verbose=1
)

model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.1672 - sparse_categorical_accuracy: 0.6696
Epoch 1: saving model to training_1/ckpt
32/32 [==============================] - 1s 17ms/step - loss: 1.1217 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.7091 - val_sparse_categorical_accuracy: 0.7780
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4287 - sparse_categorical_accuracy: 0.8795
Epoch 2: saving model to training_1/ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.4217 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.5354 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.3004 - sparse_categorical_accuracy: 0.9100
Epoch 3: saving model to training_1/ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.2988 - sparse_categorical_accuracy: 0.9090 - val_loss: 0.5098 - val_sparse_categorical_accuracy: 0.8360
Epoch 4/10
25/32 [================

In [7]:
os.listdir(checkpoint_dir)

['ckpt.index', 'checkpoint', 'ckpt.data-00000-of-00001']

In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Untrained model, accuracy: {acc * 100}%")

32/32 - 0s - loss: 2.3224 - sparse_categorical_accuracy: 0.1030 - 312ms/epoch - 10ms/step
Untrained model, accuracy: 10.300000011920929%


In [9]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Loaded model, accuracy: {acc * 100}%")

32/32 - 0s - loss: 0.3959 - sparse_categorical_accuracy: 0.8760 - 178ms/epoch - 6ms/step
Loaded model, accuracy: 87.59999871253967%


In [10]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

BATCH_SIZE = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 5 * BATCH_SIZE
)

model = create_model()

In [11]:
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs = 50,
    batch_size = BATCH_SIZE,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [12]:
os.listdir(checkpoint_dir)

['cp-0030.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0020.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001']

In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [20]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model, accuracy: {acc * 100}%")

32/32 - 0s - loss: 0.4970 - sparse_categorical_accuracy: 0.8770 - 223ms/epoch - 7ms/step
Restored model, accuracy: 87.69999742507935%


In [21]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print(f"Restored model, accuracy: {acc * 100}%")

32/32 - 0s - loss: 0.4970 - sparse_categorical_accuracy: 0.8770 - 219ms/epoch - 7ms/step
Restored model, accuracy: 87.69999742507935%


In [22]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1622 - sparse_categorical_accuracy: 0.6610
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4098 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2840 - sparse_categorical_accuracy: 0.9280
Epoch 4/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2053 - sparse_categorical_accuracy: 0.9450
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1482 - sparse_categorical_accuracy: 0.9670


In [23]:
ls saved_model

my_model/


In [24]:
ls saved_model/my_model

assets/  keras_metadata.pb  saved_model.pb  variables/


In [26]:
new_model = tf.keras.models.load_model("saved_model/my_model")

new_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)

print(f"Restored model, accuracy: {acc * 100}%")

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4142 - sparse_categorical_accuracy: 0.8690 - 187ms/epoch - 6ms/step
Restored model, accuracy: 86.90000176429749%
(1000, 10)


In [29]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1205 - sparse_categorical_accuracy: 0.6960
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4346 - sparse_categorical_accuracy: 0.8800
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2769 - sparse_categorical_accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2080 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1534 - sparse_categorical_accuracy: 0.9730


In [30]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 512)               401920    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print(f"Restored model, accuracy: {acc * 100}%")

32/32 - 0s - loss: 0.4217 - sparse_categorical_accuracy: 0.8620 - 184ms/epoch - 6ms/step
Restored model, accuracy: 86.19999885559082%
